In [60]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import os
import psycopg2

#Carpetas donde se encuentran los archivos

retail_data_path = 'C:/Users/GERARDITO/OneDrive - ASUS/Python/distributor_data_cleaner/Input/'

input_ingram = os.path.join(retail_data_path, 'INGRAM.xlsx')

input_ingram_stock = os.path.join(retail_data_path, 'INGRAM STOCK.xlsx')

input_intcomex = os.path.join(retail_data_path, 'INTCOMEX.xlsx')

input_intcomex_stock = os.path.join(retail_data_path, 'INTCOMEX STOCK.xlsx')

input_nexsys = os.path.join(retail_data_path, 'NEXSYS.xlsx')

# Variable global para obtener el ultimo dataframe limpiado

df_disti_sales = pd.DataFrame()

df_disti_stock = pd.DataFrame()


In [61]:
# Conexion a database para obtener id

connection = psycopg2.connect(
    host='localhost',
    database='asus_db',
    user='postgres',
    password='GitsyLipsy6853',
    port='5432'
)

sql_command = """ SELECT * FROM importer"""

df_importer = pd.read_sql(sql_command, connection)

df_importer = df_importer.rename(columns={'id':'importer_id'})

c:\Users\GERARDITO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [62]:

def clean_nexsys():

    #Ventas

    nexsys_sales = pd.read_excel(input_nexsys)

    df = pd.DataFrame(nexsys_sales)

    df = df.rename(columns={'Date':'date',
                            'Qty':'sell_out_units',
                            'Revenue':'sell_out_value',
                            'TAX-ID':'buyer_code',
                            'P#':'sku'})

    df['importer_name'] = 'Nexsys'

    df = df[df['sku'].notna()]

    df = df[['date',
            'importer_name',
            'sku',
            'sell_out_units',
            'sell_out_value',
            'buyer_code']]

    #stock

    nexsys_stock = pd.read_excel(input_nexsys, sheet_name="Inventory")

    df1 = pd.DataFrame(nexsys_stock)

    df1 = df1.rename(columns={'On-Hand':'stock_units',
                                'P#':'sku'})

    #Domingo

    day_s = df['date'].iloc[0].strftime("%d-%m-%Y")
    dt1 = dt.strptime(day_s, '%d-%m-%Y')
    start = dt1 - timedelta(days=dt1.weekday())
    end_week = start + timedelta(days=6)

    df1['date'] = end_week

    df1['importer_name'] = 'Nexsys'

    df1['stock_value'] = 0

    df1 = df1[['date',
                'importer_name',
                'sku',
                'stock_units',
                'stock_value']]

    global df_disti_stock

    global df_disti_sales
    
    df = pd.merge(df,df_importer[['importer_name','importer_id']],left_on='importer_name', right_on='importer_name', how='inner')

    df1 = pd.merge(df1,df_importer[['importer_name','importer_id']],left_on='importer_name', right_on='importer_name', how='inner')

    df_disti_sales = df

    df_disti_stock = df1
    

    

In [63]:
clean_nexsys()
print(df_disti_stock)

         date importer_name              sku  stock_units  stock_value  \
0  2022-10-16        Nexsys  90NB0TS1-M002V0            4            0   
1  2022-10-16        Nexsys  90NB0VD1-M00400            1            0   
2  2022-10-16        Nexsys  90NB0XD1-M003S0           28            0   
3  2022-10-16        Nexsys  90NX01X1-M15210            1            0   
4  2022-10-16        Nexsys  90NX01X1-M15220            1            0   
5  2022-10-16        Nexsys  90NX02N1-M00HU0           77            0   
6  2022-10-16        Nexsys  90NX03R1-M004J0           23            0   
7  2022-10-16        Nexsys  90NX03S1-M01100           46            0   
8  2022-10-16        Nexsys  90NX0421-M02EE0          144            0   
9  2022-10-16        Nexsys  90NX0441-M25930           25            0   
10 2022-10-16        Nexsys  90NX0460-P00110           32            0   
11 2022-10-16        Nexsys  90NX05D1-M000T0          132            0   
12 2022-10-16        Nexsys  90NX05D1-